<a href="https://colab.research.google.com/github/SammyDMartin/CC/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Uncomment for Colab
"""
!pip install "ray[rllib]" torch
!pip install nashpy
!pip install pickle5
!git clone https://github.com/SammyDMartin/CC.git
!cd CC
"""

'\n!pip install "ray[rllib]" torch\n!pip install nashpy\n!pip install pickle5\n!git clone https://github.com/SammyDMartin/CC.git\n!cd CC\n'

In [9]:
from cc_search import assess_agent_vs_others

In [14]:
from cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer
from cc_tools import maximin_policy,ideal_selfplay_policy,random_agent

opponents = [(DQNTorchPolicy,DQNTrainer),(PPOTorchPolicy,PPOTrainer),(A3CTorchPolicy,A3CTrainer)]
strategies = [maximin_policy,ideal_selfplay_policy,random_agent]

In [ ]:
res1, res_full = assess_agent_vs_others(agent_trainer=(DQNTorchPolicy,DQNTrainer),opponents=opponents, strategies=strategies,steps=2)